In [487]:
using LinearAlgebra
#make sure to cite this package
using SPICE
using Downloads: download 
using DifferentialEquations

In [488]:
#const LSK = "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls"
#const SPK = "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp"

"https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp"

In [489]:
# Download kernels
#Time Kernel
#download(LSK, "naif0012.tls")

#JPL planetary and lunar ephermis de440
#Contains the orbit of all the planets
# from years 1549 to 2650 

#download(SPK, "de440.bsp")

In [490]:
# Load leap seconds kernel
furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/naif0012.tls")

In [491]:
# Convert the calendar date to ephemeris seconds past J2000
et = utc2et("2018-02-06T20:45:00")

5.712219691849319e8

In [492]:
# Load a planetary ephemeris kernel
furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/de440.bsp")

In [ ]:
#gravitational parameters for the bodies (km3/s2)

#moon 
μ_m = 4902.80058214776

#earth
μ_e = 398600.432896939

#sun
μ_s = 132712440017.987

In [ ]:
#units used to normalize
#also in km and seconds

#distance
l_star = 385692.50

#time
t_star = 377084.152667038

#in km
lunar_radius = 1738.2 

In [ ]:
#get the jacobian that maps from rotating frame to inertial frame
function get_transformation(time)

    #state of the moon (position and velocity) relative to Earth
    moon_state = spkezr("moon",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_moon = moon_state[1:3]

    #velocity of the moon relative to Earth
    v_moon = moon_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_moon/norm(r_moon)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_moon, v_moon)/norm(cross(r_moon, v_moon))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)

    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    C = [x̃ ỹ z̃]

    #instantanous angular velocity
    #θ_dot = norm(cross(r_moon, v_moon))/(norm(r_moon)^2)

    θ_dot = (cross(r_moon, v_moon)/(norm(r_moon)^2))[3]


    return C, θ_dot

end

In [ ]:
C,_ = get_transformation(et)

In [ ]:
#transform the controls from the CR3BP units to inertial frame 

#U is a matrix of the controls you want to simulate in the ephemeris model 
function transform_controls(U, dt)

    #every control is associated with a timestep
    N = size(U)[2]

    U_ephem = zeros(3, N)

    time = et

    for i=1:N

        C, _ = get_transformation(time)

        U_ephem[:,i] = C*U[:,i] 

        time += dt

    end

    return U_ephem

end

In [ ]:
#transformation between CR3BP and ephemeris J2000 frame centered at the moon 

#x -> state in the CR3BP units
#t -> time
#et is the start time

function cr3bp_to_ephem(x, t)

    time = et + t

    println(time)

    C, θ_dot = get_transformation(time)

    C̄ = [θ_dot.*C[:,2] -θ_dot.*C[:,1]]


    #transformation matrix
    C̃ = [C zeros(3,3); C̄ zeros(3) C]

    #apply the rotation to get the state in the inertial frame
    x_ephem = C̃*x

    return x_ephem

end

    

In [ ]:
#solve using DifferentialEquations.jl
#the t here is somewhere from 0-period
function ephem_dynamics!(du, u, p, t)
    
    #du[1:6] = ephemeris_model_EarthMoon(u[1:6], t)
    #scaled version
    du[1:6] = scaled_ephemeris_dynamics(u[1:6], t)
    
    
end

In [ ]:
#gets the state and for the entire solution

function get_state(solution)
    
    N = size(solution.u)[1]

    all_states = zeros(6, N)

    for i=1:N
        all_states[:,i] = solution.u[i][1:6]
    end
    
    #all states and all stm are functions of t
    #solution.t is the time
    return all_states
end

In [ ]:
#integrate just the dynamics
#start pose and length of the integration

function just_dynamics_integrate(x_0, period)
    
    tspan = (0.0, period)
    prob = ODEProblem(ephem_dynamics!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-12, reltol=1e-12)
    
    return sol
    
end

In [ ]:
function ephemeris_model_EarthMoon(x, t)

    xdot = zeros(6)

    xdot[1:3] = x[4:6]

    #position of the spacecraft relative to the earth 
    r_qi = x[1:3]

    time = et + t

    print("TIME: ", time)

    #get positions of moon and sun relative to earth 
    rqj_moon = spkpos("moon", time, "J2000", "none", "earth")[1]
    rqj_sun = spkpos("sun", time, "J2000", "none", "earth")[1]
    
    #println("rqj moon: ", rqj_moon)
    rij_moon = rqj_moon- r_qi 
    rij_sun = rqj_sun-r_qi

    #r̈_qi - vectors that define the position of spacecraft (P_i)
    #relative to the central body (in this case the earth)

    #just Earth and Moon
    xdot[4:6] = (-μ_e/(norm(r_qi))^3)*r_qi +
                μ_m.*((rij_moon/norm(rij_moon)^3)-(rqj_moon/norm(rqj_moon)^3));




    #\ splits up into multiple lines (including the sun)
    # xdot[4:6] = (-μ_e/(norm(r_qi))^3)*r_qi + 
    #               μ_m.*((rij_moon/norm(rij_moon)^3)-(rqj_earth/norm(rqj_moon)^3)) +
    #             μ_s.*((rij_sun/norm(rij_sun)^3)-(rqj_sun/norm(rqj_sun)^3));


    return xdot

end

In [ ]:
#the x is scaled here as well as the t 

function scaled_ephemeris_dynamics(x,t)

    q_original = zeros(eltype(x),3)
    v_original = zeros(eltype(x),3)

    q_original = x[1:3]*l_star 
    v_original = x[4:6]*(l_star/t_star)
    t_original = t/t_star

    x_original = [q_original; v_original]

    #original is in the CR3BP units
    ẋ_original = zeros(eltype(x),6)

    #calculate the original xdot (no scaling)

    ẋ_original = ephemeris_model_EarthMoon(x_original, t_original)
    
    #then scale the output
    v_scaled = ẋ_original[1:3]/(l_star/t_star)
    
    a_scaled = ẋ_original[4:6]/(l_star/(t_star)^2)

    ẋ_scaled = [v_scaled; a_scaled]

    return ẋ_scaled

end

In [ ]:
#Relative to the moon

# function ephemeris_model_EarthMoon(x, t)

#     xdot = zeros(6)

#     xdot[1:3] = x[4:6]

#     #position of the spacecraft relative to the moon 
#     r_qi = x[1:3]

#     time = et + t

#     rqj_earth = spkpos("earth", time, "J2000", "none", "moon")[1]
#     rqj_sun = spkpos("sun", time, "J2000", "none", "moon")[1]
    
#     rij_earth = rqj_earth- r_qi 
#     rij_sun = rqj_sun-r_qi

#     #r̈_qi - vectors that define the position of spacecraft (P_i)
#     #relative to the central body (in this case the moon)

#     #just Earth and Moon
#     xdot[4:6] = (-μ_m/(norm(r_qi))^3)*r_qi + 
#                 μ_e.*((rij_earth/norm(rij_earth)^3)-(rqj_earth/norm(rqj_earth)^3));




#     #\ splits up into multiple lines (including the sun)
#     # xdot[4:6] = (-μ_m/(norm(r_qi))^3)*r_qi + 
#     #             μ_e.*((rij_earth/norm(rij_earth)^3)-(rqj_earth/norm(rqj_earth)^3)) + 
#     #             μ_s.*((rij_sun/norm(rij_sun)^3)-(rqj_sun/norm(rqj_sun)^3));


#     return xdot

# end

In [ ]:
L_cr = 3.850e5 #in km - distance between centers of m1 and m2
V_cr = 1.025 #in km/s - orbital velocity of m1
T_cr = 2.361e6 #in seconds - orbital period of m1 and m2
time_scale = T_cr/(2*pi)

In [ ]:
L_nasa = 389703
T_nasa = 382981

In [ ]:
#periodic orbit initial condition in the CR3BP

# x0_test = [1.1201297302380415,
#  0.0,
#  0.0059396759100811495,
#  0.0,
#  0.17677819141944426,
#  0.0]

# #in cr3bp units 
# T_periodic = 3.414975413662902


#from https://ssd.jpl.nasa.gov/tools/periodic_orbits.html

#in normalized units
x0_test = [1.1808985497899205E+0, -2.5444988241150091E-26, 1.0295054075242347E-4, 3.3765359485568778E-15, -1.5585631393981156E-1, 5.5263881873244218E-18]

T_periodic = 3.4155308065628454E+0

 #have to dimensionalize the CR3BP variables before getting simulated with the ephemeris model
x0_dim = [x0_test[1:3]*L_nasa; x0_test[4:6]*(L_nasa/T_nasa)]


#convert cr3bp state to ephemeris 
x_test_ephem = cr3bp_to_ephem(x0_dim, 0)


T_periodic_s = T_periodic*time_scale

In [ ]:
x_test_ephem_scaled = [x_test_ephem[1:3]/l_star; x_test_ephem[4:6]/(l_star/t_star)]

T_ephem_scaled = T_periodic_s/t_star

In [ ]:
sol = just_dynamics_integrate(x_test_ephem_scaled, T_ephem_scaled)

all_states = get_state(sol)

In [ ]:
using Plots
plot(all_states[1,:]*l_star, all_states[2,:]*l_star, all_states[3,:]*l_star) 
scatter!([all_states[1,1]*l_star], [all_states[2,1]*l_star], [all_states[3,1]*l_star])
#scatter!([0], [0], [0])

In [ ]:
# Get the position of Mars at `et` w.r.t. Earth

#arguments of this function
#spkpos(targ, et, ref, abcorr, obs)
#targ: target body name
#et: observer epoch
#ref: reference frame of output position vector
#abcorr: Aberration correction flag
#obs: Observing body name

#test = spkpos("mars_barycenter", et, "J2000", "none", "earth")

In [ ]:
# Get the state (km and km/s) of Mars at `et` w.r.t. Earth
#spkezr("mars_barycenter", et, "J2000", "none", "earth")

In [ ]:
rv_sun_eci, = spkezr("Sun",et,"J2000","NONE","Earth") # [km] and [km/s] position and velocity of the Sun in the ECI frame

In [ ]:
rv_sun_eci = spkezr("Sun",et,"J2000","NONE","Earth")[1][1:3]

In [ ]:
#I think the current problem is the integration is not good bc of the units